In [1]:
import pandas as pd

## Cargar datos

In [2]:
df = pd.read_csv('./data/hurto_a_residencia.csv')

In [3]:
df.head()

,fecha_hecho,cantidad,latitud,longitud,sexo,edad,estado_civil,grupo_actor,actividad_delictiva,parentesco,...,lugar,sede_receptora,bien,categoria_bien,grupo_bien,modelo,color,permiso,unidad_medida,fecha_ingestion
0,2017-01-01T13:51:00.000-05:00,1.0,6.251238,-75.586712,Mujer,38,Casado(a),Sin dato,Sin dato,Sin dato,...,"Hotel, motel y hostal",Laureles,Accesorios prendas de vestir,Prendas de vestir y accesorios,Mercancía,-1,Oro,Sin dato,Sin dato,2025-03-06T04:05:02.000-05:00
1,2017-01-01T13:51:00.000-05:00,1.0,6.251238,-75.586712,Mujer,38,Casado(a),Sin dato,Sin dato,Sin dato,...,"Hotel, motel y hostal",Laureles,Electrodoméstico video y audio y accesorios,Electrodomésticos,Mercancía,-1,Negro,Sin dato,Sin dato,2025-03-06T04:05:02.000-05:00
2,2017-01-01T13:51:00.000-05:00,1.0,6.251238,-75.586712,Mujer,38,Casado(a),Sin dato,Sin dato,Sin dato,...,"Hotel, motel y hostal",Laureles,Celular,Tecnología,Mercancía,-1,Negro,Sin dato,Sin dato,2025-03-06T04:05:02.000-05:00
3,2017-01-01T00:00:00.000-05:00,1.0,6.262758,-75.593850,Hombre,62,Soltero(a),Sin dato,Sin dato,Sin dato,...,"Hotel, motel y hostal",Laureles,Bicicleta,Vehículos de 2 o 4 ruedas,Vehículo,-1,Sin dato,Sin dato,Sin dato,2025-03-06T04:05:02.000-05:00
4,2017-01-01T00:00:00.000-05:00,1.0,6.280006,-75.564721,Mujer,53,Soltero(a),Sin dato,Sin dato,Sin dato,...,"Hotel, motel y hostal",Aranjuez,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,-1,Sin dato,Sin dato,Sin dato,2025-03-06T04:05:02.000-05:00


In [4]:
df.keys()

Index(['fecha_hecho', 'cantidad', 'latitud', 'longitud', 'sexo', 'edad',
       'estado_civil', 'grupo_actor', 'actividad_delictiva', 'parentesco',
       'ocupacion', 'discapacidad', 'grupo_especial', 'medio_transporte',
       'nivel_academico', 'testigo', 'conducta', 'modalidad',
       'caracterizacion', 'conducta_especial', 'arma_medio', 'articulo_penal',
       'categoria_penal', 'nombre_barrio', 'codigo_barrio', 'codigo_comuna',
       'lugar', 'sede_receptora', 'bien', 'categoria_bien', 'grupo_bien',
       'modelo', 'color', 'permiso', 'unidad_medida', 'fecha_ingestion'],
      dtype='object')

## Exploración de los datos

In [5]:
df['nombre_barrio'].unique()

array(['Naranjal', 'Los Colores', 'Bermejal-Los Alamos', 'Los Ángeles',
       'La Florida', 'Media Luna', 'Belén', 'Córdoba', 'Aures No.2',
       'La Loma de los Bernal', 'Los Balsos No.1', 'Santa Fé', 'Castilla',
       'Bolivariana', 'La Piñuela', 'El Tesoro', 'Berlin', 'Las Granjas',
       'Pedregal', 'Rosales', 'Las Acacias', 'Las Lomas No.1',
       'El Danubio', 'San Germán', 'El Rincón', 'Los Conquistadores',
       'Tejelo', 'Los Alpes', 'La Palma', 'San Miguel',
       'Nueva Villa del Aburrá', 'Villa Lilliam', 'Miraflores',
       'Área Urbana Cgto. San Antonio de Prado', 'Carpinelo', 'La Rosa',
       'El Nogal-Los Almendros', 'Cristo Rey', 'Loreto', 'Alejandría',
       'Altos del Poblado', 'La Pradera', 'Laureles', 'La Gloria',
       'Francisco Antonio Zea', 'El Poblado', 'La Salle',
       'Juan XXIIIi la Quiebra', 'Barrio Caycedo', 'Versalles No.2',
       'Boston', 'Jesús Nazareno', 'Llanaditas', 'Alfonso López',
       'Playón de los Comuneros', 'La Esperanza No.2'

## Creación base de datos



In [6]:
from sqlalchemy import create_engine, text

# Cambia los parámetros según tu entorno
user = "postgres"
password = "1111"
host = "localhost"
port = "5440"
database = "postgres" # Base de datos por defecto

# Crear conexión
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Comprobación básica
with engine.connect() as conn:
    print("✅ Conectado a PostgreSQL/PostGIS:", conn)

✅ Conectado a PostgreSQL/PostGIS: <sqlalchemy.engine.base.Connection object at 0x10a94fe00>


In [7]:
create_tables_sql = """
-- Asegurar extensión PostGIS
CREATE EXTENSION IF NOT EXISTS postgis;

-- Tabla de comunas
CREATE TABLE IF NOT EXISTS comuna (
    id_comuna SMALLINT PRIMARY KEY,
    nombre_comuna VARCHAR(100) NOT NULL
);

-- Tabla de barrios
CREATE TABLE IF NOT EXISTS barrio (
    id_barrio SERIAL PRIMARY KEY,
    nombre_barrio VARCHAR(100) UNIQUE NOT NULL,
    codigo_barrio VARCHAR(50) UNIQUE NOT NULL,
    codigo_comuna SMALLINT NOT NULL REFERENCES comuna(id_comuna)
);

-- Tabla de modalidades de hurto
CREATE TABLE IF NOT EXISTS modalidad (
    id_modalidad SERIAL PRIMARY KEY,
    descripcion VARCHAR(100) UNIQUE NOT NULL
);

-- Tabla de lugares donde ocurre el hurto
CREATE TABLE IF NOT EXISTS lugar (
    id_lugar SERIAL PRIMARY KEY,
    descripcion VARCHAR(100) UNIQUE NOT NULL
);

-- Tabla de grupos de bienes
CREATE TABLE IF NOT EXISTS grupo_bien (
    id_grupo SERIAL PRIMARY KEY,
    descripcion VARCHAR(100) UNIQUE NOT NULL
);

-- Tabla de categorías de bienes
CREATE TABLE IF NOT EXISTS categoria_bien (
    id_categoria SERIAL PRIMARY KEY,
    descripcion VARCHAR(100) UNIQUE NOT NULL,
    id_grupo INT REFERENCES grupo_bien(id_grupo)
);

-- Tabla de bienes específicos
CREATE TABLE IF NOT EXISTS bien (
    id_bien SERIAL PRIMARY KEY,
    descripcion VARCHAR(150) UNIQUE NOT NULL,
    id_categoria INT REFERENCES categoria_bien(id_categoria)
);
"""

In [8]:
hechos_sql = """
-- Tabla principal: hechos delictivos
CREATE TABLE IF NOT EXISTS hecho_delictivo (
    id_hecho SERIAL PRIMARY KEY,
    fecha_hecho TIMESTAMP NOT NULL,
    cantidad SMALLINT,
    ubicacion geometry(Point, 4326),
    id_barrio INT REFERENCES barrio(id_barrio),
    id_modalidad INT REFERENCES modalidad(id_modalidad),
    id_lugar INT REFERENCES lugar(id_lugar),
    id_bien INT REFERENCES bien(id_bien),
    fecha_ingestion TIMESTAMP
);

-- Índice espacial para acelerar consultas geográficas
CREATE INDEX IF NOT EXISTS idx_hecho_geom
ON hecho_delictivo
USING GIST (ubicacion);
"""

In [9]:
# Ejecutar los scripts
with engine.connect() as conn:
    conn.execute(text(create_tables_sql))
    conn.execute(text(hechos_sql))
    conn.commit()

print("✅ Tablas creadas correctamente en la base de datos.")

✅ Tablas creadas correctamente en la base de datos.


## Cargar datos

In [10]:
def insert_unique_values(conn, table, column, values):
    """Inserta valores únicos evitando duplicados."""
    for v in sorted(set(values)):
        if pd.notna(v):
            conn.execute(text(f"""
                INSERT INTO {table} ({column})
                VALUES (:val)
                ON CONFLICT ({column}) DO NOTHING;
            """), {"val": str(v).strip()})

In [11]:
with engine.begin() as conn:
    # ===============================================================
    # --- COMUNA ---
    # ===============================================================
    conn.execute(text("DELETE FROM comuna;"))
    comunas = (
        df["codigo_comuna"]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
    )

    for c in sorted(comunas):
        # Inserta solo si es número, maneja 'Sin dato'
        if c.replace(".", "").isdigit():
            conn.execute(
                text("""
                    INSERT INTO comuna (id_comuna, nombre_comuna)
                    VALUES (:id, :nombre)
                    ON CONFLICT DO NOTHING;
                """),
                {"id": int(float(c)), "nombre": f"Comuna {c}"},
            )
        else:
            conn.execute(
                text("""
                    INSERT INTO comuna (id_comuna, nombre_comuna)
                    VALUES (0, 'Sin dato')
                    ON CONFLICT DO NOTHING;
                """)
            )

    # ===============================================================
    # --- BARRIO ---
    # ===============================================================
    conn.execute(text("DELETE FROM barrio;"))
    barrios = (
        df[["nombre_barrio", "codigo_barrio", "codigo_comuna"]]
        .dropna(subset=["nombre_barrio"])
        .drop_duplicates()
    )

    for _, row in barrios.iterrows():
        codigo_comuna = str(row["codigo_comuna"]).strip()
        id_comuna = 0
        if isinstance(codigo_comuna, str) and codigo_comuna.replace(".", "").isdigit():
            id_comuna = int(float(codigo_comuna))

        conn.execute(
            text("""
                INSERT INTO barrio (nombre_barrio, codigo_barrio, codigo_comuna)
                VALUES (:nombre, :codigo, :comuna)
                ON CONFLICT (nombre_barrio) DO NOTHING;
            """),
            {
                "nombre": str(row["nombre_barrio"]).strip(),
                "codigo": str(row["codigo_barrio"]).replace("#", "").strip(),
                "comuna": id_comuna,
            },
        )

    # ===============================================================
    # --- MODALIDAD ---
    # ===============================================================
    modalidades = (
        df["modalidad"]
        .dropna()
        .astype(str)
        .str.strip()
        .replace("", "Sin dato")
        .unique()
    )

    conn.execute(text("DELETE FROM modalidad;"))
    for m in sorted(set(modalidades)):
        conn.execute(
            text("""
                INSERT INTO modalidad (descripcion)
                VALUES (:desc)
                ON CONFLICT (descripcion) DO NOTHING;
            """),
            {"desc": m},
        )

    # ===============================================================
    # --- LUGAR ---
    # ===============================================================
    lugares = (
        df["lugar"]
        .dropna()
        .astype(str)
        .str.strip()
        .replace("", "Sin dato")
        .unique()
    )

    conn.execute(text("DELETE FROM lugar;"))
    for l in sorted(set(lugares)):
        conn.execute(
            text("""
                INSERT INTO lugar (descripcion)
                VALUES (:desc)
                ON CONFLICT (descripcion) DO NOTHING;
            """),
            {"desc": l},
        )

    # ===============================================================
    # --- GRUPO_BIEN ---
    # ===============================================================
    grupos = (
        df["grupo_bien"]
        .dropna()
        .astype(str)
        .str.strip()
        .replace("", "Sin dato")
        .unique()
    )

    conn.execute(text("DELETE FROM grupo_bien;"))
    for g in sorted(set(grupos)):
        conn.execute(
            text("""
                INSERT INTO grupo_bien (descripcion)
                VALUES (:desc)
                ON CONFLICT (descripcion) DO NOTHING;
            """),
            {"desc": g},
        )

    # ===============================================================
    # --- CATEGORIA_BIEN ---
    # ===============================================================
    conn.execute(text("DELETE FROM categoria_bien;"))
    categorias = (
        df[["categoria_bien", "grupo_bien"]]
        .dropna(subset=["categoria_bien"])
        .drop_duplicates()
    )

    for _, row in categorias.iterrows():
        categoria = str(row["categoria_bien"]).strip()
        grupo = str(row["grupo_bien"]).strip()

        conn.execute(
            text("""
                INSERT INTO categoria_bien (descripcion, id_grupo)
                VALUES (
                    :desc,
                    (SELECT id_grupo FROM grupo_bien WHERE descripcion = :grupo LIMIT 1)
                )
                ON CONFLICT (descripcion) DO NOTHING;
            """),
            {"desc": categoria, "grupo": grupo},
        )

    # ===============================================================
    # --- BIEN ---
    # ===============================================================
    conn.execute(text("DELETE FROM bien;"))
    bienes = (
        df[["bien", "categoria_bien"]]
        .dropna(subset=["bien"])
        .drop_duplicates()
    )

    for _, row in bienes.iterrows():
        bien = str(row["bien"]).strip()
        cat = str(row["categoria_bien"]).strip()

        conn.execute(
            text("""
                INSERT INTO bien (descripcion, id_categoria)
                VALUES (
                    :desc,
                    (SELECT id_categoria FROM categoria_bien WHERE descripcion = :cat LIMIT 1)
                )
                ON CONFLICT (descripcion) DO NOTHING;
            """),
            {"desc": bien, "cat": cat},
        )

In [12]:
from tqdm import tqdm

with engine.begin() as conn:
    conn.execute(text("DELETE FROM hecho_delictivo;"))
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            conn.execute(text("""
                INSERT INTO hecho_delictivo (
                    fecha_hecho, cantidad, ubicacion,
                    id_barrio, id_modalidad, id_lugar, id_bien, fecha_ingestion
                ) VALUES (
                    :fecha, :cant,
                    ST_SetSRID(ST_MakePoint(:lon, :lat), 4326),
                    (SELECT id_barrio FROM barrio WHERE nombre_barrio = :barrio LIMIT 1),
                    (SELECT id_modalidad FROM modalidad WHERE descripcion = :mod LIMIT 1),
                    (SELECT id_lugar FROM lugar WHERE descripcion = :lug LIMIT 1),
                    (SELECT id_bien FROM bien WHERE descripcion = :bien LIMIT 1),
                    :ing
                );
            """), {
                "fecha": row["fecha_hecho"],
                "cant": row["cantidad"] if pd.notna(row["cantidad"]) else 1,
                "lon": row["longitud"],
                "lat": row["latitud"],
                "barrio": str(row["nombre_barrio"]).strip(),
                "mod": str(row["modalidad"]).strip(),
                "lug": str(row["lugar"]).strip(),
                "bien": str(row["bien"]).strip(),
                "ing": row["fecha_ingestion"]
            })
        except Exception:
            pass  # omite registros con referencias faltantes

100%|██████████| 42673/42673 [00:17<00:00, 2476.99it/s]


In [13]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM hecho_delictivo;"))
    print("✅ Registros cargados en hecho_delictivo:", result.scalar())

    for table in ["comuna", "barrio", "modalidad", "lugar", "grupo_bien", "categoria_bien", "bien"]:
        count = conn.execute(text(f"SELECT COUNT(*) FROM {table};")).scalar()
        print(f"📘 {table}: {count} filas")

✅ Registros cargados en hecho_delictivo: 42673
📘 comuna: 22 filas
📘 barrio: 333 filas
📘 modalidad: 26 filas
📘 lugar: 49 filas
📘 grupo_bien: 5 filas
📘 categoria_bien: 47 filas
📘 bien: 353 filas
